In [64]:
from typing import List
from ClusterParameters import ClusterParameters
from DipoleOscillation import DipoleOscillation
from Oscillation import Oscillation
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import os.path
import pickle

from SecHarmOscillation import SecHarmOscillation
from freqFinder import FreqFinder
import losses as loss

%matplotlib qt


In [65]:
nu = 0.0035
r0 = 0.068
epsD = 2.891
epsInf = 1
beta = 0.2
N = 500

wmin = 0.46
wmax = 0.48
Nw = 20
w = np.linspace(wmin, wmax, Nw)



In [66]:
params = ClusterParameters(nu, r0, epsD, epsInf)
oscillations: List[Oscillation] = []
oscillations.append(SecHarmOscillation(N, 0, params, beta))
oscillations.append(DipoleOscillation(N, params))
oscillations.append(SecHarmOscillation(N, 2, params, beta))

filename = "./savedResults/" + \
    loss.generateFilename(nu, r0, epsD, epsInf, beta, N, Nw, wmin, wmax)
if not os.path.isfile(filename):
    losses: List[np.ndarray] = []
    for osc in oscillations:
        losses.append(loss.getLosses(osc, w))
    with open(filename, "wb") as handle:
        pickle.dump((w, losses), handle)
else:
    print("Loading saved results...")
    with open(filename, "rb") as handle:
        w, losses = pickle.load(handle)


Loading saved results...


In [67]:
colors = ["g", "r", "b"]
plt.figure(1)
plt.cla()
ff = FreqFinder(params)
for i, l in enumerate(losses):
    plt.plot(w, l, colors[i])
    resFreq = ff.getResocnanceFrequencies(
        oscillations[i].multipoleN0, 10).real
    if isinstance(oscillations[i], SecHarmOscillation):
        resFreq /= 2.
    plt.scatter(resFreq, np.zeros(10), c=colors[i], marker="x")
plt.plot(w, sum(losses), 'y')
plt.xlim((wmin, wmax))
plt.grid()
plt.show()

In [68]:
print(np.min(losses[0]))

-0.5265927599045025


In [69]:
from numpy import pi
from NonlinearSources import NonlinearSources
ns = NonlinearSources(parameters=params,beta=0.2)


w1 = 0.466
Qo = oscillations[0]
r = Qo.r

psi = Qo.getPsi(w1)
rho = Qo.getRho(w1)
phi  = Qo.getPhi(w1)
phiex = ns.phiFunctions[0](r,w1*2)

rhoc = rho.conj()

q = psi*rhoc

plt.figure(2)
plt.cla()
qex = phiex*rhoc*params.r0**2
qrho = 4*np.pi*np.abs(rhoc)*params.r0**2
qphi = phi*rhoc
ax = plt.subplot()
ax.plot(r,q.real,'r')
ax.plot(r,qex.real,'k--')
ax.plot(r,qrho.real,'g--')
ax.plot(r,qphi.real,'b--')
plt.show()


plt.figure(3)
plt.cla()

psi1 = (4*pi*rho+phiex)*params.r0**2 + phi
ax = plt.subplot()
ax.plot(r,psi1.real,'r',r,psi1.imag,'b')
ax.plot(r,psi.real,'m--',r,psi.imag,'c--')
q1 = psi1*rhoc
ax.plot(r,q1.real,'g--',r,q1.imag,'k--')

plt.show()
print(np.trapz(q1*r**2,r))

(-18.04129486034586-0.34683261799231313j)
